In [1]:
tst = "Lorem ipsum dolor sit <b>amet</b>, <i>consetetur</i> sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet."

In [2]:
from  reportlab.lib.styles import ParagraphStyle as PS
from reportlab.lib.styles import getSampleStyleSheet
from  reportlab.platypus import PageBreak, Table, TableStyle
from  reportlab.platypus.paragraph import Paragraph
from  reportlab.platypus.doctemplate import PageTemplate, BaseDocTemplate
from  reportlab.platypus.tableofcontents import TableOfContents
from  reportlab.platypus.frames import Frame
from  reportlab.lib.units import cm
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from reportlab.graphics.shapes import *
from reportlab.pdfgen import canvas
import pandas as pd
from reportlab.lib.sequencer import setSequencer, Sequencer
from reportlab.pdfbase.pdfmetrics import stringWidth
from reportlab.lib.enums import TA_JUSTIFY, TA_RIGHT, TA_CENTER, TA_LEFT
import textwrap
    
setSequencer(Sequencer())

In [3]:
# dummy data
df = pd.DataFrame({'num _legss': [2, 4, 8, 0],
                   'num_wings': [2, 0, 0, 0],
                   'num_specimen_seen': [1000000, 2, 1, 8]},
                  index=['falcon', 'dog', 'spider', 'fish'])

In [4]:
# convert df to report table
def create_report_table(df):
    
    # Adjust max length (optional)
    max_len = 5 # max len for all cells
    df.columns = ["\n".join(textwrap.wrap(i, max_len, break_long_words=True)) for i in df.columns]
    df = df.applymap(lambda x: "\n".join(textwrap.wrap(str(x), max_len, break_long_words=True)))
    df.index = df.index.map(lambda x: "\n".join(textwrap.wrap(str(x), max_len, break_long_words=True)))

    # convert to list w/ sublists for report
    data = [['']+[i for i in df.columns]]
    for i, row in df.iterrows():
        vals = row.values.tolist()
        data.append([i]+vals)
    return data

In [5]:
# creates header
def header(canvas, doc):
    canvas.saveState()
    canvas.setLineWidth(.5)
    canvas.line(300,800,A4[0] - 66,800)
    header_str = "This is a test document"
    canvas.setFont('Helvetica',12)
    text_width = stringWidth(header_str, 'Helvetica',12)
    height = 805
    canvas.drawString((A4[0] - 66-text_width),height, header_str)
    canvas.restoreState()

# used by footer for total page count
class PageCount:
    tot_pages = 0
    
    @classmethod
    def get_tot_pages(cls, current_page):
        if current_page > cls.tot_pages:
            cls.tot_pages = current_page
        return cls.tot_pages
    
# creates footer
def footer(canvas, doc):
    if doc.page == 1:
        return
    tot_pages = PageCount.get_tot_pages(doc.page)
    canvas.saveState()
    #canvas.setFont('Times-Roman',9)
    #canvas.drawString(cm, 0.75 * cm, "Page %d " % doc.page)
    canvas.restoreState()
    page = f"Page {doc.page-1} of {tot_pages-1}"
    x = 128
    canvas.saveState()
    canvas.setStrokeColorRGB(0, 0, 0)
    canvas.setLineWidth(0.5)
    canvas.line(66, 60, A4[0] - 66, 60)
    canvas.setFont('Times-Roman', 10)
    canvas.drawString(A4[0]-x, 40, page)
    canvas.restoreState()
    canvas.setTitle("cool pdf")

In [37]:
class MyDocTemplate(BaseDocTemplate):
    def __init__(self, filename, **kw):
        self.allowSplitting = 0
        BaseDocTemplate.__init__(self, filename)
        template = PageTemplate('normal', 
                                [Frame(2.5*cm, 2.5*cm, 15*cm, 25*cm, id='F1')],
                                onPage=header, 
                                onPageEnd=footer)
        self.addPageTemplates(template)

    def afterFlowable(self, flowable):
        "Registers TOC entries."
        if flowable.__class__.__name__ == 'Paragraph':
            text = flowable.getPlainText()
            style = flowable.style.name
            if style == 'Heading1':
                level = 0
            elif style == 'Heading2':
                level = 1
            else:
                return
            E = [level, text, self.page-1]
            #if we have a bookmark name append that to our notify data
            bn = getattr(flowable,'_bookmarkName',None)
            if bn is not None: E.append(bn)
            self.notify('TOCEntry', tuple(E))

centered = PS(name = 'centered',
    fontSize = 30,
    leading = 16,
    alignment = 1,
    spaceAfter = 20)

data = create_report_table(df)
t=Table(data)
t.setStyle(TableStyle([('TEXTCOLOR',(0,0),(4,0),colors.red),
                       ('TEXTCOLOR',(0,0),(0,4),colors.blue),
                       ('INNERGRID', (0,0), (-1,-1), 0.25, colors.black),
                       ('BOX', (0,0), (-1,-1), 0.25, colors.black),
                       ('LINEABOVE', (0,1), (-1,-1), 0.25, colors.black),
                       ('LINEBELOW', (0,-1), (-1,-1), 2, colors.green),
                       ('ALIGN', (0, 7), (0, 7), "LEFT"),
                       ('VALIGN',(0,0),(-1,-1),'MIDDLE')
                      ]))

styles = getSampleStyleSheet()
h1 = styles['Heading1']


h2 = PS(name = 'Heading2',
    fontSize = 12,
    leading = 14)

# Create an instance of TableOfContents. Override the level styles (optional)
# and add the object to the story

toc = TableOfContents()
toc.levelStyles = [
    PS(fontName='Times-Bold', 
       fontSize=15, 
       name='TOCHeading1', 
       leftIndent=20, 
       firstLineIndent=-20, 
       spaceBefore=10, 
       leading=10),
    PS(fontSize=10, name='TOCHeading2', leftIndent=40, firstLineIndent=-20, spaceBefore=5, leading=12, wordWrap = 'CJK')
    ]
toc.dotsMinLevel = 1

# creates heading with bookmark => clickable in toc
# bn: bookmarkname, sty: style
def doHeading(text,sty):
    from hashlib import sha1
    #create bookmarkname
    bn=sha1(text.encode('utf-8')+sty.name.encode('utf-8')).hexdigest()
    print("bookmark name", bn)
    #modify paragraph text to include an anchor point with name bn
    h=Paragraph(text+'<a name="%s"/>' % bn,sty)
    #store the fbookmark name on the flowable so afterFlowable can see this
    h._bookmarkName=bn
    story.append(h)

In [38]:
# Build story.
story = []

story.append(Paragraph('<b>Table of contents</b>', centered))
story.append(toc)

story.append(PageBreak())
doHeading('First heading', h1)

story.append(Paragraph("<b>"+"TITLE" + "</b>"+"<br></br><br></br>" + tst, PS("body",
                       spaceBefore=15,  # Space to Text above
                       spaceAfter=15,
                       leftIndent = 10,  # since border needs space too
                       rightIndent = 10,                       fontSize=10,
                       textColor=colors.black,
                       borderColor=colors.black,
                       borderWidth=.5, 
                       borderPadding=5,  # space b/w text and border
                       backColor=colors.Color(red=(240.0/255),green=(240.0/255),blue=(240.0/255)), 
                       borderRadius=5)))
doHeading('First sub heading', h2)
story.append(Paragraph('Text in first sub heading', PS('body', fontSize=5)))
story.append(PageBreak())
doHeading('Second sub heading', h2)
story.append(Paragraph('Text in second sub heading', PS('body', fontSize=9, alignment=TA_LEFT)))
story.append(t)
story.append(PageBreak())
doHeading('Last heading', h1)
story.append(Paragraph('Text in last heading', PS('body', fontSize=9)))
doc = MyDocTemplate('test_doc.pdf')
doc.multiBuild(story)  # multi build makes several passes, needed bc of toc

bookmark name 03c9db24ce32b5b9e6ca549ac017aa4e08c7ce69
bookmark name 7e75303101d6a4dde29e228e26d4babb0843d18c
bookmark name a6dd7024b00393112373f367671c2164701f8397
bookmark name 32d876f1d8e3fb287a95908edf2544816062ea12


2